In [1]:
# Gerekli kütüphaneler
!pip install timm scikit-learn numpy pandas tqdm torch torchvision

# VMamba reposunu klonla
# Not: Bu adım, eğer Colab ortamında modeliniz yüklü değilse gereklidir.
# VMamba_v10.pth modelini manuel olarak Drive'a yüklediyseniz bu satıra gerek kalmayabilir.
!git clone https://github.com/MzeroMiko/VMamba.git

Cloning into 'VMamba'...
remote: Enumerating objects: 8910, done.
remote: Counting objects: 100% (1398/1398), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 8910 (delta 1341), reused 1223 (delta 1223), pack-reused 7512 (from 2)
Receiving objects: 100% (8910/8910), 34.84 MiB | 46.09 MiB/s, done.
Resolving deltas: 100% (5492/5492), done.


In [2]:
import os
from google.colab import drive

# Bu kısım sadece Colab'da çalışır
try:
    print("Google Drive'a bağlanılıyor...")
    drive.mount('/content/drive')
    print("Drive bağlantısı başarılı.")

    # Veri setinizin Google Drive'daki yolu
    # Lütfen 'new_data' klasörünün doğru yolda olduğundan emin olun
    DATA_DIR = '/content/drive/MyDrive/data_v1'

except ImportError:
    # Colab dışında çalışıyorsanız bu yol kullanılır
    print("Colab ortamı algılanmadı. Yerel yollar kullanılıyor.")
    DATA_DIR = './data_v1'

# Veri klasörünün varlığını kontrol et
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Hata: '{DATA_DIR}' yolu bulunamadı. Lütfen Google Drive'daki veri klasörünün doğru konumda olduğundan emin olun.")
else:
    print(f"Veri klasörü başarıyla bulundu: {DATA_DIR}")

Google Drive'a bağlanılıyor...
Mounted at /content/drive
Drive bağlantısı başarılı.
Veri klasörü başarıyla bulundu: /content/drive/MyDrive/data_v1


In [3]:
import os
import random
import json
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import time
from datetime import datetime

# --- YARDIMCI FONKSİYON: SÜRE FORMATLAMA ---

def format_duration(seconds):
    """Saniye cinsinden süreyi (H saat, M dakika, S saniye) formatına dönüştürür.
       İstenen HH saat, MM dakika, SS saniye formatı kullanılır."""
    seconds = int(round(seconds))
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02d} saat, {m:02d} dakika, {s:02d} saniye"

# --- YAPILANDIRMA VE YOL TANIMLARI ---

# LÜTFEN KONTROL EDİN: Drive üzerindeki verilerinizin ana klasör yolu
ROOT_DIR = '/content/drive/MyDrive/data_v1'
# EĞİTİLMİŞ MODELLERİN KAYDEDİLECEĞİ KLASÖR YOLU
DRIVE_MODEL_DIR = '/content/drive/MyDrive/vmamba_models'

# SADECE TÜRKÇE (TR) İÇİN HARF LİSTESİ
language_info = {
    # Bu liste alt klasör isimlerinizle eşleşmelidir.
    'TR': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z'],
}
ALL_LANG_CODES = ['TR']

# Eğitim parametreleri
NUM_EPOCHS = 50 # Colab'da GPU'da bile uzun sürebilir
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
TEST_SIZE = 0.2
RANDOM_STATE = 42

# --- MODEL VE VERİ SINIFLARI ---

class CustomMambaClassifier(nn.Module):
    """VMamba'ya benzer şekilde çalışan, ConvNeXt tabanlı görüntü sınıflandırıcısı."""
    def __init__(self, num_classes, pretrained=True):
        super(CustomMambaClassifier, self).__init__()
        # Güçlü bir temel olarak ConvNeXt Base kullanılıyor.
        self.backbone = timm.create_model('convnext_base', pretrained=pretrained, features_only=True)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.num_classes = num_classes

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.backbone.feature_info[-1]['num_chs'], self.num_classes)
        )

    def forward(self, x):
        # Yalnızca son özellik haritası kullanılır
        x = self.backbone(x)[-1]
        x = self.avgpool(x)
        x = self.head(x)
        return x

class SignLanguageDataset(Dataset):
    """İşaret dili görüntüleri için özel veri kümesi sınıfı."""
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# --- VERİ HAZIRLAMA FONKSİYONU ---

def prepare_lang_data(root_dir, lang_code, test_size=TEST_SIZE, random_state=RANDOM_STATE):
    """Sadece belirtilen dilin resimlerini toplar ve etiketler."""
    all_image_paths = []
    all_labels = []

    lang_path = os.path.join(root_dir, lang_code)
    if not os.path.exists(lang_path):
        print(f"Uyarı: '{lang_code}' diline ait klasör bulunamadı. Bu dil atlanıyor.")
        return None, None, None, None, None

    # Bu döngü, tek el resimlerini içeren klasörleri (örn: C) ve
    # çift el resimlerini içeren klasörleri (örn: H) ayırt etmeksizin etiketleriyle toplar.
    for char_dir in language_info.get(lang_code, []):
        char_path = os.path.join(lang_path, char_dir)
        if os.path.isdir(char_path):
            label = char_dir
            for img_name in os.listdir(char_path):
                img_path = os.path.join(char_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    all_image_paths.append(img_path)
                    all_labels.append(label)

    print(f"\nVeri Kontrolü: '{lang_code}' için Toplam {len(all_labels)} resim bulundu.")
    if len(all_labels) == 0:
        print(f"Hata: '{lang_code}' dizininde hiç resim bulunamadı. Atlanıyor.")
        return None, None, None, None, None

    le = LabelEncoder()
    encoded_labels = le.fit_transform(all_labels)

    # Veri setini eğitim ve test olarak bölme
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_image_paths, encoded_labels, test_size=test_size, random_state=random_state, stratify=encoded_labels
    )

    # Sınıf haritasını oluştur ve kaydet
    class_map = {name: int(label) for name, label in zip(le.classes_, le.transform(le.classes_))}
    class_map_path = os.path.join(DRIVE_MODEL_DIR, f'class_map_{lang_code}.json')
    os.makedirs(os.path.dirname(class_map_path), exist_ok=True)
    try:
        with open(class_map_path, 'w', encoding='utf-8') as f:
            json.dump(class_map, f, ensure_ascii=False, indent=4)
            print(f"\n{lang_code} için {len(class_map)} sınıf haritası '{class_map_path}' dosyasına kaydedildi.")
    except Exception as e:
        print(f"Uyarı: Sınıf haritası kaydedilemedi: {e}")


    return train_paths, test_paths, train_labels, test_labels, le.classes_

# Veri zenginleştirme (Data Augmentation) ve normalizasyon adımları
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Doğrulama/Test için kullanılan transform
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- MODEL DEĞERLENDİRME ---

def evaluate_model(model, test_loader, device, class_names):
    """Verilen modelin test verisi üzerindeki performansını değerlendirir."""
    model.eval()
    all_preds = []
    all_labels = []

    test_bar = tqdm(test_loader, desc="[Test]", leave=False)
    with torch.no_grad():
        for images, labels in test_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, zero_division=0)

    return accuracy, report

# --- TEK DİL EĞİTİM FONKSİYONU ---

def train_single_language_model(lang_code, train_paths, test_paths, train_labels, test_labels, class_names):
    """Tek bir dil için modeli eğitir, kaydeder ve raporlar."""

    print(f"\n{'='*20} {lang_code} DİLİ İÇİN EĞİTİM BAŞLIYOR ({len(class_names)} Sınıf) {'='*20}")

    epoch_log_path = os.path.join(DRIVE_MODEL_DIR, f'vmamba_{lang_code}_epoch_details.txt')

    train_dataset = SignLanguageDataset(train_paths, train_labels, transform)
    test_dataset = SignLanguageDataset(test_paths, test_labels, val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    num_classes = len(class_names)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = CustomMambaClassifier(num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    # Log dosyası için başlık yazma
    with open(epoch_log_path, 'w', encoding='utf-8') as f:
        f.write(f"--- VMAMBA EĞİTİM DETAYLI EPOCH LOGLARI ({lang_code}) ---\n")
        f.write(f"Başlangıç Tarih/Saat: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Veri Seti Boyutu (Eğitim): {len(train_dataset)}, (Test): {len(test_dataset)}\n")
        f.write(f"Toplam Sınıf Sayısı: {num_classes}\n")
        f.write("-" * 50 + "\n\n")


    # --- EĞİTİM DÖNGÜSÜ (COLAB'DA UZUN SÜRE BİLİR!) ---
    for epoch in range(NUM_EPOCHS):
        epoch_start_time = time.time()
        epoch_start_datetime = datetime.now().strftime("%H:%M:%S")

        model.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [{lang_code} Eğitimi]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_bar.set_postfix({'Kayıp': running_loss / (train_bar.n + 1)})

        avg_train_loss = running_loss / len(train_loader)

        # --- EPOCH SONU DEĞERLENDİRME ---

        overall_accuracy, overall_report = evaluate_model(model, test_loader, device, class_names)

        epoch_end_time = time.time()
        epoch_end_datetime = datetime.now().strftime("%H:%M:%S")
        epoch_duration = epoch_end_time - epoch_start_time

        # Süreyi HH saat, MM dakika, SS saniye formatına dönüştür
        formatted_epoch_duration = format_duration(epoch_duration)

        # --- MODEL VE LOG KAYDETME ---

        # 1. Modeli kaydet (pth)
        model_filename = f'vmamba_{lang_code}_epoch_{epoch+1}.pth'
        model_path = os.path.join(DRIVE_MODEL_DIR, model_filename)
        torch.save(model.state_dict(), model_path)
        print(f"\nModel, '{model_path}' olarak kaydedildi.")

        # 2. Detaylı Epoch Raporunu kaydet (txt)
        with open(epoch_log_path, 'a', encoding='utf-8') as f:
            f.write(f"| --- EPOCH {epoch+1}/{NUM_EPOCHS} Raporu ({lang_code}) ---\n")
            f.write(f"| Başlangıç Saati: {epoch_start_datetime}\n")
            f.write(f"| Bitiş Saati: {epoch_end_datetime}\n")
            f.write(f"| Süre: {formatted_epoch_duration}\n") # İSTENEN FORMAT
            f.write(f"| Ortalama Eğitim Kaybı: {avg_train_loss:.4f}\n")
            f.write(f"| Test Doğruluğu: {overall_accuracy:.4f}\n")
            f.write("| Sınıflandırma Raporu:\n")

            for line in overall_report.split('\n'):
                 f.write(f"| {line.strip()}\n")

            f.write("| ------------------------------------------------------------------\n\n")

        print(f"Epoch {epoch+1} tamamlandı. Doğruluk: {overall_accuracy:.4f}. Detaylı rapor '{epoch_log_path}' dosyasına eklendi.")
        print("-" * 50)


    # --- DİL BAZLI BİTİŞ RAPORU ---
    print(f"\n{lang_code} DİLİ EĞİTİMİ BAŞARIYLA TAMAMLANDI.")
    print(f"Son Model: {model_path}")


# --- ANA EĞİTİM BORU HATTI ---

def run_training_pipeline():
    """SADECE TR dili için eğitim sürecini yönetir."""

    global_start_time = time.time()
    os.makedirs(DRIVE_MODEL_DIR, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Kullanılan Cihaz: {device}")

    all_language_stats = {}

    # Sadece TR için eğitim başlatılıyor
    for lang_code in ALL_LANG_CODES:
        lang_start_time = time.time()

        train_paths, test_paths, train_labels, test_labels, class_names = prepare_lang_data(ROOT_DIR, lang_code)

        if train_paths is None:
            continue

        train_single_language_model(lang_code, train_paths, test_paths, train_labels, test_labels, class_names)

        lang_duration = time.time() - lang_start_time
        all_language_stats[lang_code] = format_duration(lang_duration)

        print(f"\n--- {lang_code} İşlemleri Tamamlandı. Toplam Süre: {all_language_stats[lang_code]} ---\n")


    # --- GLOBAL BİTİŞ RAPORU ---
    global_duration = time.time() - global_start_time
    formatted_global_duration = format_duration(global_duration)

    print("\n\n" + "="*70)
    print("TÜRKÇE DİL MODELİNİN EĞİTİMİ BAŞARIYLA TAMAMLANDI.")
    print("="*70)
    print(f"Tüm Eğitim Süresi: {formatted_global_duration}")

# Eğitimi başlat
if __name__ == '__main__':
    run_training_pipeline()


Kullanılan Cihaz: cuda

Veri Kontrolü: 'TR' için Toplam 34500 resim bulundu.

TR için 23 sınıf haritası '/content/drive/MyDrive/vmamba_models/class_map_TR.json' dosyasına kaydedildi.

==================== TR DİLİ İÇİN EĞİTİM BAŞLIYOR (23 Sınıf) ====================


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

KeyboardInterrupt: 